<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/Transform1/Transformer_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Модель Transformer-1

## Доставляем библиотеки

In [46]:
!pip install transformers transformers[sentencepiece] sentencepiece datasets sacremoses corus sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c898c04871c116ca0d5e11d6aa99bb54bcdb31a0336b8543cbb988ccd0890294
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


## Испортируем общие библиотеки

In [23]:
import pandas as pd
import numpy as np
import torch

from tqdm import tqdm

tqdm.pandas()

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Дополнительные функции

In [118]:
def translate(text):
  """
    Функция перевода
  """
  input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
  outputs = model.generate(input_ids).to(DEVICE)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

def rnd_df(df, count, field_name):
  """
    Взять рандомные значения из датасета
  """
  rnd = np.random.randint(1000, df.shape[0])
  return df.iloc[rnd, field_name]

def similar(embeddings, sentence, n=1):
  """
    Получить похожие вложения
  """
  s_emb =  model.encode([sentence], convert_to_tensor=True).to(DEVICE)
  cos_scores = util.cos_sim(s_emb, embeddings).to(DEVICE)
  cpu_cos_scores = cos_scores.tolist()
  top_results = np.argpartition(cpu_cos_scores[0], -n)[-n:]
  
  return cos_scores, top_results

## 1. Возмем предобученную трансформерную архитектуру и решим задачу перевода

In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

In [20]:
from datasets import load_dataset

dataset = load_dataset("cosmos_qa")

Using custom data configuration default
Reusing dataset cosmos_qa (/root/.cache/huggingface/datasets/cosmos_qa/default/0.1.0/e4e873eafb86b174fbfdf309a8baac27525753e655fe93f48f99023efa950ae6)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
df = pd.DataFrame(dataset["train"])
df.head()

,id,context,question,answer0,answer1,answer2,answer3,label
0,3Q9SPIIRWJKVQ8244310E8TUS6YWAC##34V1S5K3GTZMDU...,Good Old War and person L : I saw both of thes...,"In the future , will this person go to see oth...",None of the above choices .,This person likes music and likes to see the s...,This person only likes Good Old War and Person...,Other Bands is not on tour and this person can...,1
1,3E24UO25QZOMYXHZN4TEH9EMT9GO6L##3UN61F00HXNWYQ...,I mean it : not one person said ANYTHING to me...,Why might have the temp agency tell me I am no...,The company hiring the temp workers might have...,The temp agency hiring the temp workers might ...,None of the above choices .,I might have had a change of mind working for ...,0
2,3M4KL7H8KVL125AYH2V35D1E0A016T##3SB5N7Y3O426ET...,Leaving my shift Thursday day shift I arrived ...,What may have caused the radio to erupt with d...,My partner needed a medic unit .,Someone was running from the ambulances after ...,None of the above choices .,Someone was running from the cops and got into...,3
3,3D5G8J4N5CI2K40F4RZLF9OG2L4VTJ##34MAJL3QP721EU...,"So , last day in Seattle , and my flight was a...",Why did I chit chat with my old manager ?,Because my flight was at 1:30 .,Because I left Seattle feeling really good and...,Because it 's my last day in Seattle .,Because I enjoy talking to him .,3
4,3MQKOF1EE428I44N8B42W7P86DKDW4##3NJM2BJS4ZLBGN...,Ms. Mumma informed Mr. Dail that evidence had ...,Why did I burst into tears ?,Because Ms. Mumma was informative .,Because I had an intense emotional response to...,Because Mr. Dail was informed .,Because I fell out of my chair .,1


In [25]:
df['context_ru'] = (df.iloc[:10, :])['context'].progress_apply(np.vectorize(translate))

100%|██████████| 10/10 [00:18<00:00,  1.81s/it]


In [40]:
for key, row in df[:10].iterrows():
  print(f"eng: {row['context']}\nrus: {row['context_ru']}\n")

eng: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .
rus: "Добрая старая война и человек L": Я видела обе группы в среду вечером, и они оба сбили меня с толку. "Хорошая Старая Война" - это акустика и заставляет меня улыбаться. Я действительно не могу помочь, но быть счастливой, когда я слушаю их; я думаю, что это тот факт, что они казались такими счастливыми, когда играли.

eng: I mean it : not one person said ANYTHING to me . They would have if I did something wrong , right ? But , this morning , I got a call from my guy at the temp agency and he said that I was no longer needed in that position , that I did n't need to go out there .
rus: Я имею в виду, что ни один человек не сказал мне ничего, если бы я сделал что-то не так, но сегодня ут

## Попробуем реализовать метод поиска ближайших статей из датасета **gazeta.csv.gz**


In [42]:
!wget -O gazeta.csv.gz -qq --no-check-certificate https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

In [44]:
from corus import load_ods_gazeta

records = load_ods_gazeta('gazeta.csv.gz')
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [45]:
df = pd.DataFrame(records)
df.head()

,0,1,2,3,4,5,6,7
0,2012-09-27 10:54:51,https://www.gazeta.ru/science/news/2012/09/27/...,None,Наука,None,Японские физики повторили синтез 113-го элемента,Японские ученые из физического центра RIKEN за...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
1,2015-12-01 15:28:45,https://www.gazeta.ru/army/news/7957079.shtml,None,Армия,None,Times: Россия строит новую авиабазу в Сирии,Россия готовится расширить свою военную операц...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
2,2013-04-18 16:42:06,https://www.gazeta.ru/travel/news/2013/04/18/n...,None,Новости,None,Власти Египта гарантируют безопасность российс...,Египет ждет российских туристов и гарантирует ...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
3,2012-09-26 13:54:13,https://www.gazeta.ru/politics/news/2012/09/26...,None,Политика,None,Гордума Новочеркасска приняла отставку мэра го...,В Ростовской области депутаты гордумы Новочерк...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."
4,2012-09-26 21:11:33,https://www.gazeta.ru/culture/news/2012/09/26/...,None,Культура,None,"Министерство культуры назвало фильмы, на котор...",Министерство культуры России определило перече...,"Stats(fb=None, vk=None, ok=None, twitter=None,..."


In [125]:
for idx in range(2):
  print(idx)

0
1


In [126]:
from sentence_transformers import SentenceTransformer, util

model_name = "clips/mfaq"

model = SentenceTransformer(model_name)

sentences = df[6][:1000]

for idx in range(2):
  print("="*10)
  print(f"Generate {idx + 1}")
  print("="*10)

  embeddings = model.encode(df[6][:1000], convert_to_tensor=True).to(DEVICE)
  sentence = rnd_df(df, 1000, 6)
  
  cos_scores, top_results = similar(embeddings, sentence, 5)

  for top in sentences[top_results.tolist()]:
    print(f"{top}\n")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Generate 1
В школу №263 в столичном районе Отрадное ворвался старшеклассник с винтовкой. Подросток убил учителя и полицейского, после чего взял в заложники детей. К месту происшествия стянуты силы полиции. Cтрелка-школьника уже задержали и передали следователям. Правоохранители сообщают, что это отличник с хорошими характеристиками, который хотел отомстить учителю географии. «Газета.Ru» ведет онлайн-трансляцию из Отрадного.nn«Газета.Ru» завершает онлайн-трансляцию. В ближайшее время читайте материал корреспондента «Газеты.Ru» Максима Солопова, который работал на месте происшествия.nnnУ погибшего при исполнении своих обязанностей старшего сержанта 38-летнего Сергея Бушуева остались жена и дочь, сообщает пресс-служба ГУ МВД России по Москве. В органах внутренних дел работал с 1994 года. Руководство и личный состав столичного главка выражают глубокие соболезнования и обещают его семье оказать необходимую помощь.nnПресс-секретарь президента Дмитрий Песков сообщил в эфире «Русской службы но

## Вывод

По **первому пункту** можно сказать что перевод довольно таки занимательный, но не без своих недостатков. По **второму пункту** более менее похожи тексты, но если выборку делать небольшую, при более большой выборке стоит задуматься об улучшении.